In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

from analysis_variables import *

import myPIDselector

import pandas as pd
import seaborn as sns

In [ ]:
start = time.time()

# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'

##josie laptop 
#topdir = "/Users/josieswann/Desktop/important documents"

# On Bellis' laptop
#topdir = '/home/bellis/babar_data/bnv_plambda/'

# At Bellis' home
topdir = '/home/bellis/babar_data/bnv_plambda'

# On Bellis' laptop
#topdir = './'

filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'
#filename = f'{topdir}/Background_and_signal_SP_modes_All_runs.parquet'

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")
IS_MC=True


#'''
# Collision data
#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Data_Only_Run_1_BLINDED.parquet'
#filename = f'{topdir}/Data_All_runs_BLINDED.parquet'
#filename = f'{topdir}/Data_All_runs_BLINDED.parquet'

start = time.time()

data_collision = ak.from_parquet(filename)
#data_collision = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")

print(type(data_collision))
#'''


In [ ]:
### information about cross section --> what we'll use to calculate scaling values for histograms 

dataset_information = pd.read_csv("dataset_statistics.csv")
cs_data= pd.read_csv("SP_cross_sections_and_labels.csv")

no_notes= cs_data.drop(["Uncertainty","Note: cross sections found at https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/Cross_sections,_luminosities,_and_other_vital_stats"], axis= 1)
no_notes

In [ ]:
sp = data['spmode']

np.unique(sp.to_list())

In [ ]:
region_definitions

In [ ]:
#bat.fill_new_entry_with_tag_side_B(data)
#data['BtagSideMes']



In [ ]:
# Make our histograms
all_hists = bat.create_empty_histograms(hist_defs)

bkg_spmodes = ['998', '1005', '1235', '1237', '3981']
sig_spmodes = ['-999']

spmodes = bkg_spmodes + sig_spmodes

weights = {}
for sp in spmodes:
    weights[sp] = bat.scaling_value(int(sp), dataset_information=dataset_information, cs_data=cs_data, plot=False, verbose=False)
    #weights[sp] = 1

### bat.scaling_value is in Babar_analysis_tools.py 

# Scale the signal higher
weights['-999'] = 1000
weights['0'] = 1

print(weights)
print()
print(spmodes)

In [ ]:
# Need to get the original duplicates mask for any other cuts we might generate outside the function
dcuts = bat.get_final_masks(data, region_definitions=region_definitions)

print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')



In [ ]:
###############################################################################
# Make the masks
#mask_event = dcuts[3]['event']
#mask_event = dcuts[4]['event']
mask_event = dcuts[1]['event']
#mask_event = dcuts[-1]['event']
#mask_event = dcuts[2]['event'] & dcuts[3]['event'] & dcuts[4]['event']

spmode = '998'

mask_sp = data['spmode']==spmode

#mask = mask_event & mask_sp
mask = mask_sp

# Make the plot
mes =    ak.flatten(data[mask]['BpostFitMes'])
DeltaE = ak.flatten(data[mask]['BpostFitDeltaE'])

bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True)

#print(len(mes),len(DeltaE))

## Other HISTOGRAMS 
all_hists = bat.create_empty_histograms(hist_defs)

x = ak.flatten(data[mask]['Lambda0_unc_Mass'])
weight = weights[spmode]
all_hists['Lambda0_unc_Mass'].fill(var=x, SP= spmode, cuts= f"{0}", weight= weight)

plt.figure()
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="fill", color='red', label= spmode)
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="step", color='black')
plt.legend();

# pyhf

https://pyhf.github.io/pyhf-tutorial/HelloWorld.html

https://pyhf.github.io/pyhf-tutorial/IntroToHiFa.html


## A one-bin example

https://pyhf.readthedocs.io/en/v0.7.6/

## Dummy 2D thing

https://pyhf.readthedocs.io/en/v0.7.6/examples/notebooks/multiBinPois.html

## 2D distributions

https://stackoverflow.com/questions/66038864/2d-distributions-in-the-histfactory

Maybe this thread?


https://github.com/scikit-hep/pyhf/discussions/2070

Use `ravel`

Drawing on some work I did here

https://colab.research.google.com/drive/1J4IQ8DDkKKQPxQH2A06l1Z350YlWqTL7?usp=sharing


## My old stackoverflow question

https://stackoverflow.com/questions/64257667/trying-to-put-together-a-teaching-example-with-pyhf

## BELLE question that might be relevant

Has some discussions about how 0-bins are handled.

https://github.com/scikit-hep/pyhf/discussions/2070


## Multiple backgrounds

https://pyhf.readthedocs.io/en/v0.7.6/examples/notebooks/multichannel-coupled-histo.html

## Toy simulations

https://pyhf.readthedocs.io/en/v0.7.6/examples/notebooks/learn/TestStatistics.html

## Raveling / unravel

Return to original shape

https://stackoverflow.com/questions/52046347/unravel-1d-list-back-to-3d-array


In [ ]:
import pyhf

import json

import matplotlib.pyplot as plt
import numpy as np
from pyhf.contrib.viz import brazil

import logging
import json
import math
import numpy as np
import matplotlib.pyplot as plt

import pyhf
from pyhf import Model, optimizer
from pyhf.simplemodels import uncorrelated_background
from pyhf.contrib.viz import brazil

from scipy.interpolate import griddata
import scrapbook as sb



In [ ]:
def draw_plot_dm(mu=1, bins=[], observed=[], background=[], signal=[]):
    #bins = [5, 55, 550]
    #observed = [60, 43, 40]
    #background = [58.0, 42.5, 36.0]
    #signal = [i * mu for i in [2, 4, 3]]

    print(f"signal: {signal}")
    print(f"background: {background}")
    print(f"observed: {observed}\n")

    width = bins[1] - bins[0]

    fig, ax = plt.subplots()
    ax.bar(bins, background, width=width, label=r"Background", edgecolor="red", alpha=0.5)
    ax.bar(
        bins,
        signal,
        width=width,
        label=r"Signal",
        edgecolor="blue",
        bottom=background,
        alpha=0.5,
    )
    ax.scatter(bins, observed, color="black", label="Observed")
    #ax.set_ylim(0, 6)
    ax.legend()
    ax.set_xlabel(r"$p_T$ [GeV/c]", fontsize=12)
    ax.set_ylabel("Count", fontsize=12);


# Fitting BaBar data

### References

https://pyhf.github.io/pyhf-tutorial/HelloWorld.html

This documentation mentions how the uncertainties are set for 0 bins

https://pyhf.readthedocs.io/en/v0.7.6/likelihood.html#luminosity-lumi

"For bins in the model where:

the samples nominal expected rate is zero, or

the scale factor is zero.

nuisance parameters will be allocated, but will be fixed to 1 in the calculation (as staterror is a multiplicative modifier this results in multiplying by 1)."


In [ ]:
df = pd.read_parquet('inference_region_counts_SP998_FINAL_CUT.parquet')
Nbkg = df['N'].values

df = pd.read_parquet('inference_region_counts_SIGNAL_FINAL_CUT.parquet')
Nsig = df['N'].values

df = pd.read_parquet('inference_region_counts_COLLISION_DATA_FINAL_CUT.parquet')
Nobs = df['N'].values

print(Nobs)

In [ ]:
model = pyhf.simplemodels.uncorrelated_background(
    signal=(Nsig/15000).tolist(), bkg=(Nbkg).tolist(), bkg_uncertainty=(Nbkg).tolist()
)
model

In [ ]:
print(json.dumps(model.spec, indent=2))

In [ ]:
print(f"  channels: {model.config.channels}")
print(f"     nbins: {model.config.channel_nbins}")
print(f"   samples: {model.config.samples}")
print(f" modifiers: {model.config.modifiers}")
print(f"parameters: {model.config.parameters}")
print(f"  nauxdata: {model.config.nauxdata}")
print(f"   auxdata: {model.config.auxdata}")

In [ ]:
# 1 more than the number of bins
model.expected_data([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])


In [ ]:

model.spec['channels'][0]

In [ ]:
model.spec['channels'][0]['samples'][0]

In [ ]:
nbins = model.config.channel_nbins['singlechannel']
bins = np.linspace(0,nbins,nbins)
#obs = model.expected_data

sig = model.spec['channels'][0]['samples'][0]['data']
bkg = model.spec['channels'][0]['samples'][1]['data']

# Set some signal
Nobs[0] = 2

print(len(bins), bins)
print(len(sig), sig)
print(len(bkg), bkg)
print(len(Nobs), Nobs)
print()

#print(obs)

draw_plot_dm(mu=1, bins=bins, observed=Nobs, background=bkg, signal=sig)


In [ ]:
data = model.config.auxdata

data

In [ ]:
data = pyhf.tensorlib.astensor(Nobs.tolist() + model.config.auxdata)

print(data)


ul = pyhf.infer.intervals.upper_limits.upper_limit(data, model)


In [ ]:
c = model.config

c.auxdata

In [ ]:
c.auxdata_order

In [ ]:
c.par_names

In [ ]:
c.nmaindata

In [ ]:
c.channels

In [ ]:
len(c.channels)

# Playground

In [ ]:
#https://pyhf.github.io/pyhf-tutorial/IntroToHiFa.html

def draw_plot_dm(mu=1):
    bins = [5, 55, 550]
    observed = [60, 43, 40]
    background = [58.0, 42.5, 36.0]
    signal = [i * mu for i in [2, 4, 3]]

    print(f"signal: {signal}")
    print(f"background: {background}")
    print(f"observed: {observed}\n")

    fig, ax = plt.subplots()
    ax.bar(bins, background, width=[10.0, 90, 900], label=r"Background", edgecolor="red", alpha=0.5)
    ax.bar(
        bins,
        signal,
        width=[10.0, 90, 900],
        label=r"Signal",
        edgecolor="blue",
        bottom=background,
        alpha=0.5,
    )
    ax.scatter(bins, observed, color="black", label="Observed")
    #ax.set_ylim(0, 6)
    ax.legend()
    ax.set_xlabel(r"$p_T$ [GeV/c]", fontsize=12)
    ax.set_ylabel("Count", fontsize=12);


draw_plot_dm()

In [ ]:
# Get the data

# Make the masks
#mask_event = dcuts[3]['event']
#mask_event = dcuts[4]['event']
#mask_event = dcuts[1]['event']
mask_event = dcuts[-1]['event']
#mask_event = dcuts[2]['event'] & dcuts[3]['event'] & dcuts[4]['event']


vals = {}
for spmode in ['998', '-999']:
    
    #spmode = '998'
    
    mask_sp = data['spmode']==spmode
    
    mask = mask_event & mask_sp
    #mask = mask_sp
    
    # Make the plot
    mes =    ak.flatten(data[mask]['BpostFitMes'])
    DeltaE = ak.flatten(data[mask]['BpostFitDeltaE'])
    
    n = 100
    if n>len(mes):
        n = len(mes)
    print(n,len(mes))
    mes = np.random.choice(mes.to_numpy(), n, replace=False)
    DeltaE = np.random.choice(DeltaE.to_numpy(), n, replace=False)
    
    print(len(mes), mes[:5])
    print(len(DeltaE), DeltaE[:5])

    vals[spmode] = {"mes": mes, "DeltaE": DeltaE}

#vals

In [ ]:
plt.figure()
plt.plot(vals['998']["mes"], vals['998']['DeltaE'], '.')
plt.plot(vals['-999']["mes"], vals['-999']['DeltaE'], '.')
plt.xlabel('mES')
plt.ylabel('DeltaE')
plt.xlim(5.2, 5.3)
plt.ylim(-0.2, 0.2)
;

In [ ]:
nbinsx = 2
nbinsy = 3
mes_lo = 5.2
mes_hi = 5.3
de_lo = -0.2
de_hi =  0.2

hvals = {}

epsilon = 0.001

hvals['sig'], xedges, yedges = np.histogram2d(vals['-999']["mes"], vals['-999']["DeltaE"], bins=[nbinsx, nbinsy], range=[[mes_lo, mes_hi], [de_lo, de_hi]])
#print(hvals)
org_shape = hvals['sig'].copy()

hvals['sig'] = np.ravel(hvals['sig']+epsilon).tolist()

hvals['bkg'], xedges, yedges = np.histogram2d(vals['998']["mes"], vals['998']["DeltaE"], bins=[nbinsx, nbinsy], range=[[mes_lo, mes_hi], [de_lo, de_hi]])
#print(hvals)
hvals['bkg'] = np.ravel(hvals['bkg']+epsilon).tolist()

# IF THE UNCERTAINTIES ARE TOO SMALL, IT BREAKS DOWN  #NOPE
#hvals['bkg_uncert'] = np.sqrt(np.array(hvals['bkg'])).tolist()
hvals['bkg_uncert'] = (0.5*np.ones_like(hvals['bkg'], dtype=float)).tolist()


print(hvals)


In [ ]:
x = np.array(hvals['sig'])

np.reshape(x, org_shape.shape)

In [ ]:
print(org_shape.shape)

org_shape



In [ ]:
# Unravel / ravel example

# https://stackoverflow.com/questions/52046347/unravel-1d-list-back-to-3d-array

# raveled = np.ravel(arr)
# new_arr = raveled.reshape(*arr.shape)


In [ ]:
sig_scale_factor = 0.05

signal = (np.array(hvals['sig'])*sig_scale_factor).tolist()

model = pyhf.simplemodels.uncorrelated_background(
    signal=signal, bkg=hvals['bkg'], bkg_uncertainty=hvals['bkg_uncert']
)

print("aux data")
print(model.config.auxdata)
print("\nsignal: ",signal)
print("\nbkg   : ",hvals['bkg'])

print()
print(len(model.config.auxdata))
print(len(hvals['sig']))
print(len(hvals['bkg']))

In [ ]:
# Mock data
#observed_data = (np.array(hvals['bkg']) + np.random.normal(1,0.05,len(hvals['bkg']))).tolist()  + model.config.auxdata
#observed_data = (np.array(hvals['bkg']) + np.random.randint(0,1,len(hvals['bkg']))).tolist()  + model.config.auxdata

# Add in some signal
mock_signal = np.array(hvals['sig'])+np.random.randint(1,2,len(hvals['sig']))
mock_signal[np.array(hvals['sig'])<1] = 0 
mock_signal[np.array(hvals['sig'])>=1] = 1 

print("sig        : ",   hvals['sig'])
print("mock signal: ",   mock_signal)
observed_data = (np.array(hvals['bkg']) + np.random.randint(0,1,len(hvals['bkg'])) + mock_signal).tolist()  + model.config.auxdata

# MAKE SURE DATA IS POSITIVE
print("observed_data: ",observed_data)

In [ ]:
test_mu = 1.0

print("model")
print(model)
print()

print("observed_data")

print(observed_data)



In [ ]:
CLs_obs, CLs_exp = pyhf.infer.hypotest(
    test_mu, observed_data, model, test_stat="qtilde", return_expected=True
)

print(f"Observed: {CLs_obs}, Expected: {CLs_exp}")

In [ ]:
CLs_obs, CLs_exp = pyhf.infer.hypotest(
    test_mu,  # null hypothesis
    observed_data,
    model,
    test_stat="qtilde",
    return_expected_set=True,
)
print(f"      Observed CLs: {CLs_obs:.4f}")
for expected_value, n_sigma in zip(CLs_exp, np.arange(-2, 3)):
    print(f"Expected CLs({n_sigma:2d} σ): {expected_value:.4f}")

In [ ]:
poi_values = np.linspace(0.1, 5, 50)
obs_limit, exp_limits, (scan, results) = pyhf.infer.intervals.upper_limits.upper_limit(
    observed_data, model, poi_values, level=0.05, return_results=True
)
print(f"Upper limit (obs): μ = {obs_limit:.4f}")
print(f"Upper limit (exp): μ = {exp_limits[2]:.4f}")

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10.5, 7)
ax.set_title("Hypothesis Tests")

artists = brazil.plot_results(poi_values, results, ax=ax)

# Toys?

https://pyhf.readthedocs.io/en/v0.7.6/examples/notebooks/toys.html

In [ ]:
# mu' = 0: background-like
pars_bkg = model.config.suggested_init()
pars_bkg[model.config.poi_index] = 0.0
print(f"Background parameters: {list(zip(model.config.parameters, pars_bkg))}")

# mu' = 1: signal-like
pars_sig = model.config.suggested_init()
pars_sig[model.config.poi_index] = 1.0
print(f"Signal parameters: {list(zip(model.config.parameters, pars_sig))}")

# make the pdfs
pdf_bkg = model.make_pdf(pyhf.tensorlib.astensor(pars_bkg))
pdf_sig = model.make_pdf(pyhf.tensorlib.astensor(pars_sig))

In [ ]:
# note: pdf.sample takes in a "shape" N=(10000,) given the number of samples
n_samples = 100

# mu' = 0
mc_bkg = pdf_bkg.sample((n_samples,))
# mu' = 1
mc_sig = pdf_sig.sample((n_samples,))

print(mc_bkg.shape)
print(mc_sig.shape)

In [ ]:
mc_bkg

In [ ]:
init_pars = model.config.suggested_init()
par_bounds = model.config.suggested_bounds()
fixed_params = model.config.suggested_fixed()

qtilde_bkg = pyhf.tensorlib.astensor(
    [
        pyhf.infer.test_statistics.qmu_tilde(
            1.0, mc, model, init_pars, par_bounds, fixed_params
        )
        for mc in mc_bkg
    ]
)
qtilde_sig = pyhf.tensorlib.astensor(
    [
        pyhf.infer.test_statistics.qmu_tilde(
            1.0, mc, model, init_pars, par_bounds, fixed_params
        )
        for mc in mc_sig
    ]
)

In [ ]:
toy_calculator_qtilde = pyhf.infer.utils.create_calculator(
    "toybased",
    model.expected_data(pars_sig),
    model,
    ntoys=n_samples,
    test_stat="qtilde",
)
qtilde_sig, qtilde_bkg = toy_calculator_qtilde.distributions(1.0)

In [ ]:
qmu_bounds = model.config.suggested_bounds()
print(f"Old bounds: {qmu_bounds}")
qmu_bounds[model.config.poi_index] = (-10, 10)
print(f"New bounds: {qmu_bounds}")

In [ ]:
toy_calculator_qmu = pyhf.infer.utils.create_calculator(
    "toybased",
    model.expected_data(model.config.suggested_init()),
    model,
    par_bounds=qmu_bounds,
    ntoys=n_samples,
    test_stat="q",
)
qmu_sig, qmu_bkg = toy_calculator_qmu.distributions(1.0)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)
for ax in axes:
    ax.set_xticks(np.arange(0, 10))
ax0, ax1 = axes.flatten()

bins = np.linspace(0, 10, 26)

ax0.hist(
    qmu_sig.samples,
    bins=bins,
    histtype="step",
    density=True,
    label="$f(q_1|1)$ signal-like",
    linewidth=2,
)
ax0.hist(
    qmu_bkg.samples,
    bins=bins,
    histtype="step",
    density=True,
    label="$f(q_1|0)$ background-like",
    linewidth=2,
)
ax0.set_xlabel(r"(a) $q_1$", fontsize=18)
ax0.set_ylabel(r"$f\,(q_1|\mu')$", fontsize=18)
ax0.set_title(r"Test statistic $(q_1)$ distributions")
ax0.legend()

ax1.hist(
    qtilde_sig.samples,
    bins=bins,
    histtype="step",
    density=True,
    label=r"$f(\tilde{q}_1|1)$ signal-like",
    linewidth=2,
)
ax1.hist(
    qtilde_bkg.samples,
    bins=bins,
    histtype="step",
    density=True,
    label=r"$f(\tilde{q}_1|0)$ background-like",
    linewidth=2,
)
ax1.set_xlabel(r"(b) $\tilde{q}_1$", fontsize=18)
ax1.set_ylabel(r"$f\,(\tilde{q}_1|\mu')$", fontsize=18)
ax1.set_title(r"Alternative test statistic $(\tilde{q}_1)$ distributions")
ax1.legend()


plt.setp(axes, xlim=(0, 9), ylim=(1e-3, 2), yscale="log")
fig.set_size_inches(14, 6)
fig.tight_layout(pad=2.0)

In [ ]:
# 2D

sig = np.ravel([[12.0, 11.0], [5, 6]]).tolist()
bkg = np.ravel([[50.0, 52.0],[7, 7]]).tolist()
bkg_uncert = np.ravel([[3.0, 7.0], [1, 2]]).tolist()

print(sig)
print(bkg)
print(bkg_uncert)
print()


#'''

model = pyhf.simplemodels.uncorrelated_background(
    signal=sig, bkg=bkg, bkg_uncertainty=bkg_uncert
)

print("aux data")
print(model.config.auxdata,'\n')

observed_data = np.ravel([[51, 48], [8, 4]]).tolist() + model.config.auxdata
print("observed_data")
print(observed_data,'\n')

test_mu = 1.0

print("model")
print(model)
print()

print("observed_data")

print(observed_data)

CLs_obs, CLs_exp = pyhf.infer.hypotest(
    test_mu, observed_data, model, test_stat="qtilde", return_expected=True
)

print(f"Observed: {CLs_obs}, Expected: {CLs_exp}")

#'''

In [ ]:
# 2D - small signals

sig = np.ravel([[0.30, 0.40], [0.2, 0.1]]).tolist()
bkg = np.ravel([[1, 2],[1, 0]]).tolist()
bkg_uncert = np.ravel([[1.1, 1.0], [1, 1]]).tolist()

print(sig)
print(bkg)
print(bkg_uncert)
print()


#'''

model = pyhf.simplemodels.uncorrelated_background(
    signal=sig, bkg=bkg, bkg_uncertainty=bkg_uncert
)

print("aux data")
print(model.config.auxdata,'\n')
print(model.config.auxdata,'\n')


observed_data = np.ravel([[1, 2], [0, 1]]).tolist() + model.config.auxdata
print("observed_data")
print(observed_data,'\n')

test_mu = 1.0

print("model")
print(model)
print()

print("observed_data")

print(observed_data)

CLs_obs, CLs_exp = pyhf.infer.hypotest(
    test_mu, observed_data, model, test_stat="qtilde", return_expected=True
)

print(f"Observed: {CLs_obs}, Expected: {CLs_exp}")

#'''

# Playing with toys and upper limits
https://pyhf.github.io/pyhf-tutorial/Toys.html



In [ ]:
# Original
#'''
observed = [51, 48]
background = [50.0, 52.0]
background_uncert = [3.0, 7.0]
signal = [12.0, 11.0]
#'''

# Small numbers
#'''
observed = [0, 1]
background = [0.5, 0.5]
background_uncert = [1.0, 1.0]
signal = [0.0, 2.0]
#'''

model = pyhf.simplemodels.uncorrelated_background(
    signal=signal, bkg=background, bkg_uncertainty=background_uncert
)
observed_data = observed + model.config.auxdata
print("observed_data: ",observed_data)

test_mu = 1.0

CLs_obs, CLs_exp = pyhf.infer.hypotest(
    test_mu, observed_data, model, test_stat="qtilde", return_expected=True
)
print(f"Observed: {CLs_obs:.8f}, Expected: {CLs_exp:.8f}")

print("Hypothesis")

CLs_obs, CLs_exp = pyhf.infer.hypotest(
    1.0,  # null hypothesis
    observed + model.config.auxdata,
    model,
    test_stat="qtilde",
    return_expected_set=True,
)
print(f"      Observed CLs: {CLs_obs:.4f}")
for expected_value, n_sigma in zip(CLs_exp, np.arange(-2, 3)):
    print(f"Expected CLs({n_sigma:2d} σ): {expected_value:.4f}")

print()
print("More")
print()
CLs_obs, CLs_exp = pyhf.infer.hypotest(
    1.0,  # null hypothesis
    observed + model.config.auxdata,
    model,
    test_stat="qtilde",
    return_expected_set=True,
    calctype="toybased",
    ntoys=1_000,
    track_progress=False,
)
print(f"      Observed CLs: {CLs_obs:.4f}")
for expected_value, n_sigma in zip(CLs_exp, np.arange(-2, 3)):
    print(f"Expected CLs({n_sigma:2d} σ): {expected_value:.4f}")

In [ ]:
model = pyhf.simplemodels.uncorrelated_background(
    signal=[12.0, 11.0], bkg=[50.0, 52.0], bkg_uncertainty=[3.0, 7.0]
)
observed_data = [51, 48] + model.config.auxdata
print("observed_data: ",observed_data)

test_mu = 1.0
CLs_obs, CLs_exp = pyhf.infer.hypotest(
    test_mu, observed_data, model, test_stat="qtilde", return_expected=True
)
print(f"Observed: {CLs_obs:.8f}, Expected: {CLs_exp:.8f}")

# Hypothesis testing - low stats

https://pyhf.github.io/pyhf-tutorial/Toys.html

In [ ]:
# Small numbers
# From example
#'''
observed = [5, 7]
background = [5.0, 6.0]
background_uncert = [0.5, 1.2]
signal = [0.5, 1.0]
#'''

#'''
observed = [0, 1]
background = [0.0, 1.0]
background_uncert = [0.5, 1.2]
signal = [0.5, 1.0]
#'''



model_low = pyhf.simplemodels.uncorrelated_background(
    signal=signal, bkg=background, bkg_uncertainty=background_uncert
)

observed_data = observed + model.config.auxdata
print("observed_data: ",observed_data)

#model_low = pyhf.simplemodels.uncorrelated_background(
#    signal=[0.5, 1.0], bkg=[5.0, 6.0], bkg_uncertainty=[0.5, 1.2]
#)



CLs_obs, CLs_exp = pyhf.infer.hypotest(
    1.0,  # null hypothesis
    observed + model_low.config.auxdata,
    model_low,
    test_stat="qtilde",
    return_expected_set=True,
    calctype="asymptotics",
)
print(f"      Observed CLs: {CLs_obs:.4f}")
for expected_value, n_sigma in zip(CLs_exp, np.arange(-2, 3)):
    print(f"Expected CLs({n_sigma:2d} σ): {expected_value:.4f}")

print("\nWith toys...")
CLs_obs, CLs_exp = pyhf.infer.hypotest(
    1.0,  # null hypothesis
    observed + model_low.config.auxdata,
    model_low,
    test_stat="qtilde",
    return_expected_set=True,
    calctype="toybased",
    ntoys=1_000,
    track_progress=True,
)

print(f"      Observed CLs: {CLs_obs:.4f}")
for expected_value, n_sigma in zip(CLs_exp, np.arange(-2, 3)):
    print(f"Expected CLs({n_sigma:2d} σ): {expected_value:.4f}")

Discussions about 0-rate bins.

https://github.com/scikit-hep/pyhf/issues/293

https://github.com/scikit-hep/pyhf/issues/180


## Variable-width bins
https://stackoverflow.com/questions/66165423/pyhf-support-for-variable-bin-width-histograms

Here is a discussion of the error that I get with some 0-rate bins. 

https://stackoverflow.com/questions/60514470/minimal-pyhf-example-failing-with-inequality-constraints-incompatible

In [ ]:
model = pyhf.simplemodels.uncorrelated_background(
    signal=[5.0, 10.0], bkg=[50.0, 60.0], bkg_uncertainty=[5.0, 12.0]
)

CLs_obs, CLs_exp = pyhf.infer.hypotest(
    1.0,  # null hypothesis
    [53.0, 65.0] + model.config.auxdata,
    model,
    test_stat="qtilde",
    return_expected_set=True,
)
print(f"      Observed CLs: {CLs_obs:.4f}")
for expected_value, n_sigma in zip(CLs_exp, np.arange(-2, 3)):
    print(f"Expected CLs({n_sigma:2d} σ): {expected_value:.4f}")

In [ ]:
model = pyhf.simplemodels.uncorrelated_background(
    signal=[0.5, 1.0], bkg=[5.0, 6.0], bkg_uncertainty=[1.0, 1.0]
)

CLs_obs, CLs_exp = pyhf.infer.hypotest(
    1.0,  # null hypothesis
    [5.0, 6.0] + model.config.auxdata,
    model,
    test_stat="qtilde",
    return_expected_set=True,
)
print(f"      Observed CLs: {CLs_obs:.4f}")
for expected_value, n_sigma in zip(CLs_exp, np.arange(-2, 3)):
    print(f"Expected CLs({n_sigma:2d} σ): {expected_value:.4f}")

In [ ]:
epsilon = 0.001

model = pyhf.simplemodels.uncorrelated_background(
    # Works
    #signal=[0.5, 1.0], bkg=[1.0, 1.2], bkg_uncertainty=[0.5, 0.5]

    # Test 0-rate bin
    #signal=[0.5, 1.0], bkg=[0.0, 0.5], bkg_uncertainty=[0.5, 0.5]
    
    # Adding an epsilon offset to 0-rate
    signal=[0.5, 1.0], bkg=[0.0+epsilon, 0.5], bkg_uncertainty=[0.5, 0.5]

)

# mu is fraction of sig and bkg
# mu = 1 --> all bkg
# mu = 0 --> all sig
mu = 1.0

# Works
#observations = [1.0, 2.0]

# Test
observations = [1.0, 2.0]


CLs_obs, CLs_exp = pyhf.infer.hypotest(
    mu,  # null hypothesis
    observations + model.config.auxdata,
    model,
    test_stat="qtilde",
    return_expected_set=True,
)
print(f"      Observed CLs: {CLs_obs:.4f}")
for expected_value, n_sigma in zip(CLs_exp, np.arange(-2, 3)):
    print(f"Expected CLs({n_sigma:2d} σ): {expected_value:.4f}")

In [ ]:
poi_values = np.linspace(0.1, 5, 50)
obs_limit, exp_limits, (scan, results) = pyhf.infer.intervals.upper_limits.upper_limit(
    observations+model.config.auxdata, model, poi_values, level=0.05, return_results=True
)
print(f"Upper limit (obs): μ = {obs_limit:.4f}")
print(f"Upper limit (exp): μ = {exp_limits[2]:.4f}")

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10.5, 7)
ax.set_title("Hypothesis Tests")

artists = brazil.plot_results(poi_values, results, ax=ax)